In [ ]:
# Import libraries and standardize path
import os
from pathlib import Path
import numpy as np
import cv2
from tkinter.filedialog import askopenfilename
import SimpleITK as sitk
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt

In [ ]:
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")
# Change directory as needed

os.chdir("C:/Users/Lynna/OneDrive/Desktop/Capstone/Data")

In [ ]:
# Image Subtraction 

# Load NIfTI file
nifti_image = nib.load("ute_0_3mm_uncorrected.nii.gz")
ute_data = nifti_image.get_fdata() 
ute_data = ute_data[:, 83, :] # Access the coronal slice 83

# Convert the NumPy array to a SimpleITK image
ute_image_sitk = sitk.GetImageFromArray(ute_data)

# Convert to float32
ute_image_sitk = sitk.Cast(ute_image_sitk, sitk.sitkFloat32)

# Normalize UTE image to [0, 1]
ute_normalized = ute_image_sitk / np.max(ute_image_sitk)

# Load Processed pCT using workflow
image_path = "UTE_Bone_Mask.png" # Processed using slice 83

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
if image is None:
    raise FileNotFoundError(f"Image at path '{image_path}' could not be loaded. Check the file path.")

# Convert to SimpleITK image format and cast to float
image_sitk = sitk.GetImageFromArray(image)
image_sitk = sitk.Cast(image_sitk, sitk.sitkFloat32)
pct_normalized = image_sitk / np.max(image_sitk)

# Check that sizes match 
print(ute_image_sitk.GetSize())
print(image_sitk.GetSize())

# Display images to ensure same slice and orientation
ute_image = sitk.GetArrayFromImage(ute_image_sitk)
pct_image = sitk.GetArrayFromImage(image_sitk)

# Display results
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(ute_image, cmap="gray", aspect="auto")
plt.title("MRI Coronal Slice")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(pct_image, cmap="gray", aspect="auto")
plt.title("pCT Coronal Slice")
plt.axis("off")

plt.show()

# Subtraction
subtracted_sitk= sitk.Subtract(pct_normalized, ute_normalized)

subtracted_image = sitk.GetArrayFromImage(subtracted_sitk)

# Convert back to pixel range between 0 to 255
subtracted_image = np.uint8(255 * subtracted_image)

# Display results
plt.figure(figsize=(10, 5))
plt.imshow(subtracted_image, cmap="gray")
plt.title("Subtracted")
plt.axis("off")

plt.show()

# Save results
cv2.imwrite('Subtracted_Image', subtracted_image)